# A Bayesian Analysis of Team Performance Metrics in Major League Baseball

by Mecchia Alessandro and Sergio Fernandez Diz

## 1. Imports

In [ ]:
import pybaseball as pb
from pybaseball import statcast
import pandas as pd
from pybaseball import cache
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np
from scipy import stats
from sklearn.preprocessing import MinMaxScaler
from pybaseball import playerid_reverse_lookup
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import norm
import numpy as np
from scipy.stats import gaussian_kde
import plotly.graph_objects as go
import seaborn as sns
import base64
import codecs
import pymc as pm
import arviz as az
import os
import statsmodels.api as sm


## 2. Introduction

Baseball is a team sport in which two teams alternate between offense and defense: the goal is to hit the ball with a bat and score points by running between four bases. It is a game heavily based on strategy, individual skill, and statistics, especially in the pitcher-batter matchup. It is one of the most data-driven sports, in which every action generates detailed statistics used to evaluate performance, strategies, and probabilities of success.

### Moneyball and the Rise of Sabermetrics

In 2003, the book ```Moneyball: The Art of Winning an Unfair Game``` was published about the Oakland Athletics baseball team and its general manager Billy Beane. It describes the team's sabermetric approach to assembling a competitive baseball team on a small budget. The central premise of Moneyball is that the collective wisdom of baseball insiders (including players, managers, coaches, scouts, and executives) over the past century is outdated, subjective, and often wrong, and that the statistics traditionally used to evaluate players, such as stolen bases, runs batted in, and batting average, are relics of a 19th-century view of the game. Sabermetrics and statistical analysis have shown, for example, that on-base percentage and slugging percentage are better measures of hitting. The Oakland A's began looking for players who were “undervalued by the market,” meaning they were receiving salaries lower than their ability to contribute to wins, as measured by these advanced statistics.This approach proved to be highly effective, as the Athletics achieved sustained competitive success despite operating with one of the lowest payrolls in Major League Baseball.

Wikipedia contributors. (2024). Moneyball. [In Wikipedia, The Free Encyclopedia](https://en.wikipedia.org/wiki/Moneyball)

In this project, we use a Bayesian inference approach to analyze metrics related to the Moneyball philosophy, both at the team and individual player level. We compare traditional statistics with advanced metrics such as OBP, SLG, and OPS, evaluating their ability to explain and predict success in terms of wins. The use of Bayesian models allows us to quantify uncertainty and update performance estimates in a manner consistent with the dynamic nature of the game.

## 3. Data Collection

To obtain the datasets used in this project, we relied primarily on pybaseball, a Python library that automates data collection from major public baseball databases such as Baseball Reference, Baseball Savant, and FanGraphs. The library provides access to a wide range of data, including Statcast pitch-level data, batting and pitching statistics, team standings, awards, and other advanced metrics, both at the individual pitch level and in aggregated form over seasons or custom time periods.
In addition, team-level data for the annual period from 2020 to 2024 were manually collected from [Baseball-Reference](https://www.baseball-reference.com) to complement and validate the datasets obtained through the library.

### 3.1 Event Statcast

The statcast function retrieves pitch-level statcast data for a given date or range or dates.

In [ ]:
cache.enable()

In [ ]:
#df = statcast(start_dt='2025-03-01', end_dt='2025-10-30')
df = pd.read_parquet("data/statcast_2025.parquet")
#df.to_parquet("statcast_2025.parquet", compression="snappy")

### 3.2 Players

In [ ]:
# priors: usiamo le funzioni pitcher_stats e batting_stats per ottenere i dati delle prestazioni dei lanciatori e dei battitori
# le usiamo pure per EDA. poi utilizzeremo statcast per ottenere dati più dettagliati sulle partite specifiche
#batter_stats = pb.batting_stats_bref(2024)
#pitcher_stats = pb.pitching_stats_bref(2024)
batter_stats = pd.read_csv('data/batter_stats_2024.csv')
pitcher_stats = pd.read_csv('data/pitcher_stats_2024.csv')
#batter_stats.head()

### 3.3 Teams

## 4. Data Exploration

### 4.1 Data Description

#### Statcast dataset

**Pitch Characteristics**
| Category | Variable | Description | Level |
|---------|----------|-------------|-------|
| Pitch | pitch_type | Type of pitch thrown (e.g. FF, SL, CH) | Pitch |
| Pitch | release_speed | Pitch velocity at release (mph) | Pitch |
| Pitch | effective_speed | Effective pitch velocity | Pitch |
| Pitch | release_spin_rate | Spin rate at release (rpm) | Pitch |
| Pitch | spin_axis | Axis of spin of the pitch | Pitch |
| Pitch | release_extension | Distance from rubber at release (ft) | Pitch |


**Ball Movement and Location**
| Category | Variable | Description | Level |
|---------|----------|-------------|-------|
| Movement | pfx_x | Horizontal pitch movement (inches) | Pitch |
| Movement | pfx_z | Vertical pitch movement (inches) | Pitch |
| Location | plate_x | Horizontal location at plate | Pitch |
| Location | plate_z | Vertical location at plate | Pitch |
| Zone | zone | Strike zone location (1–14) | Pitch |

**Batted Ball and Contact**
| Category | Variable | Description | Level |
|---------|----------|-------------|-------|
| Contact | launch_speed | Exit velocity of batted ball (mph) | Pitch |
| Contact | launch_angle | Launch angle (degrees) | Pitch |
| Contact | hit_distance_sc | Estimated hit distance (feet) | Pitch |
| Contact | bb_type | Batted ball type (groundball, flyball, etc.) | Pitch |
| Contact | launch_speed_angle | Launch speed/angle classification | Pitch |


**Expected and Value Metrics**
| Category | Variable | Description | Level |
|---------|----------|-------------|-------|
| Expected | estimated_ba_using_speedangle | Expected batting average | Pitch |
| Expected | estimated_woba_using_speedangle | Expected wOBA | Pitch |
| Expected | estimated_slg_using_speedangle | Expected slugging percentage | Pitch |
| Value | woba_value | Actual wOBA value | Pitch |
| Value | iso_value | Isolated power (ISO) value | Pitch |
| Value | babip_value | Batting average on balls in play | Pitch |


**Game Context**
| Category | Variable | Description | Level |
|---------|----------|-------------|-------|
| Game | game_date | Date of the game | Game |
| Game | inning | Inning number | Game |
| Game | inning_topbot | Top or bottom of the inning | Game |
| Game | balls | Balls count | Pitch |
| Game | strikes | Strikes count | Pitch |
| Game | outs_when_up | Number of outs | Pitch |


**Player Information**
| Category | Variable | Description | Level |
|---------|----------|-------------|-------|
| Player | batter | Batter ID | Player |
| Player | pitcher | Pitcher ID | Player |
| Player | stand | Batter handedness | Player |
| Player | p_throws | Pitcher handedness | Player |
| Player | age_bat | Batter age | Player |
| Player | age_pit | Pitcher age | Player |


**Win and Run Expectancy**
| Category | Variable | Description | Level |
|---------|----------|-------------|-------|
| Expectancy | delta_run_exp | Change in run expectancy | Pitch |
| Expectancy | delta_home_win_exp | Change in home win probability | Pitch |
| Expectancy | home_win_exp | Home team win probability | Game |
| Expectancy | bat_win_exp | Batting team win probability | Game |


#### 4.1.1 Event Statcast

In [ ]:
df.info()

In [ ]:
print(df.columns.to_list())

spiegare le variabili principali che useremo, nome del battitore e del lanciatore codificati dagli ID

In [ ]:
batter_ids = df['batter'].dropna().astype(int).unique()

In [ ]:
batters = playerid_reverse_lookup(batter_ids, key_type='mlbam')

batters = batters[['key_mlbam', 'name_first', 'name_last']].copy()
batters['batter_name'] = batters['name_first'].str.title() + ', ' + batters['name_last']

batters = batters[['key_mlbam', 'batter_name']]
batters.columns = ['batter_id', 'batter_name']
batters.head(2)

In [ ]:
id_to_name = dict(zip(batters['batter_id'], batters['batter_name']))

df['batter_name'] = df['batter'].map(id_to_name)
df = df.rename(columns={"player_name": "pitcher_name"})

df.head(2)

In [ ]:
categorical_cols = {}
numerical_cols = {}

categorical_cols["df"] = df.select_dtypes(include=["object", "category", "bool"]).columns.tolist()
numerical_cols["df"]   = df.select_dtypes(include=["int64", "float64"]).columns.tolist()

#### 4.1.2 Players

##### 4.1.2.1 Pitchers

| Variable | Description |
|---------|-------------|
| Name | Player name |
| Lev | Competition level (MLB) |
| Tm | Team |
| Age | Player age |
| #days | Days on active roster |
| G | Games pitched |
| GS | Games started |
| W | Wins |
| L | Losses |
| SV | Saves |
| IP | Innings pitched |
| H | Hits allowed |
| R | Runs allowed |
| ER | Earned runs |
| BB | Walks allowed |
| SO | Strikeouts |
| HR | Home runs allowed |
| HBP | Hit batters |
| ERA | Earned run average |
| AB | At-bats against |
| 2B | Doubles allowed |
| 3B | Triples allowed |
| IBB | Intentional walks |
| GDP | Grounded into double plays |
| SF | Sacrifice flies allowed |
| SB | Stolen bases allowed |
| CS | Caught stealing |
| PO | Pickoffs |
| BF | Batters faced |
| Pit | Total pitches |
| Str | Total strikes |
| StL | Looking strikes |
| StS | Swinging strikes |
| GB/FB | Ground ball to fly ball ratio |
| LD | Line drives |
| PU | Pop-ups |
| WHIP | Walks plus hits per inning pitched |
| BAbip | Batting average on balls in play |
| SO9 | Strikeouts per 9 innings |
| SO/W | Strikeout-to-walk ratio |
| mlbID | Unique MLB player identifier |


In [ ]:
print(f"Pitcher stats:")
pitcher_stats.head(1)

##### 4.1.2.2 Batters

| Variable | Description |
|---------|-------------|
| Name | Player name |
| Lev | Competition level (MLB) |
| Tm | Team |
| Age | Player age |
| #days | Days on active roster |
| G | Games played |
| PA | Plate appearances |
| AB | At-bats |
| R | Runs scored |
| H | Hits |
| 2B | Doubles |
| 3B | Triples |
| HR | Home runs |
| RBI | Runs batted in |
| BB | Walks |
| IBB | Intentional walks |
| SO | Strikeouts |
| HBP | Hit by pitch |
| SH | Sacrifice hits |
| SF | Sacrifice flies |
| GDP | Grounded into double plays |
| SB | Stolen bases |
| CS | Caught stealing |
| BA | Batting average |
| OBP | On-base percentage |
| SLG | Slugging percentage |
| OPS | On-base plus slugging |
| mlbID | Unique MLB player identifier |


In [ ]:
print(f"Batter stats:")
batter_stats.head(1)

In [ ]:
categorical_cols["batter_stats"] = batter_stats.select_dtypes(include=["object", "category", "bool"]).columns.tolist()
numerical_cols["batter_stats"]   = batter_stats.select_dtypes(include=["int64", "float64"]).columns.tolist()
categorical_cols["pitcher_stats"] = pitcher_stats.select_dtypes(include=["object", "category", "bool"]).columns.tolist()
numerical_cols["pitcher_stats"]   = pitcher_stats.select_dtypes(include=["int64", "float64"]).columns.tolist()

print("BATTER_STATS")
print("Categoriche:", categorical_cols["batter_stats"])
print("Numeriche:", numerical_cols["batter_stats"])

print("\nPITCHER_STATS")
print("Categoriche:", categorical_cols["pitcher_stats"])
print("Numeriche:", numerical_cols["pitcher_stats"])

#### 4.1.3 Teams

##### 4.1.3.1 Team Batting

| Variable | Description |
|---------|-------------|
| #Bat | Number of batters used |
| BatAge | Average batter age |
| R/G | Runs scored per game |
| G | Games played |
| PA | Plate appearances |
| AB | At-bats |
| R | Runs scored |
| H | Hits |
| 2B | Doubles |
| 3B | Triples |
| HR | Home runs |
| RBI | Runs batted in |
| SB | Stolen bases |
| CS | Caught stealing |
| BB | Walks |
| SO | Strikeouts |
| BA | Batting average |
| OBP | On-base percentage |
| SLG | Slugging percentage |
| OPS | On-base plus slugging |
| OPS+ | Park-adjusted OPS (league average = 100) |
| TB | Total bases |
| GDP | Grounded into double plays |
| HBP | Hit by pitch |
| SH | Sacrifice hits |
| SF | Sacrifice flies |
| IBB | Intentional walks |
| LOB | Left on base |


In [ ]:
data_folder = 'data/'
team_batter_stats = pd.DataFrame()
for file in os.listdir(data_folder):
    if file.startswith('team_batter_stats_') and file.endswith('.csv'):
        year = file.split('_')[-1].split('.')[0]
        temp_df = pd.read_csv(os.path.join(data_folder, file))
        temp_df['year'] = int(year)
        team_batter_stats = pd.concat([team_batter_stats, temp_df], ignore_index=True)
team_batter_stats.head(2)

In [ ]:
# remove the rows League Average and NaN teams
team_batter_stats = team_batter_stats[(~team_batter_stats["Tm"].isin(["League Average"])) &
    (team_batter_stats["Tm"].notna())
]

##### 4.1.3.2 Team Pitching

| Variable | Description |
|---------|-------------|
| #P | Number of pitchers used |
| PAge | Average pitcher age |
| RA/G | Runs allowed per game |
| W | Wins |
| L | Losses |
| W-L% | Win–loss percentage |
| ERA | Earned run average |
| G | Games pitched |
| GS | Games started |
| GF | Games finished |
| CG | Complete games |
| tSho | Team shutouts |
| cSho | Combined shutouts |
| SV | Saves |
| IP | Innings pitched |
| H | Hits allowed |
| R | Runs allowed |
| ER | Earned runs |
| HR | Home runs allowed |
| BB | Walks allowed |
| IBB | Intentional walks |
| SO | Strikeouts |
| HBP | Hit batters |
| BK | Balks |
| WP | Wild pitches |
| BF | Batters faced |
| ERA+ | Park-adjusted ERA (league average = 100) |
| FIP | Fielding independent pitching |
| WHIP | Walks plus hits per inning pitched |
| H9 | Hits allowed per 9 innings |
| HR9 | Home runs allowed per 9 innings |
| BB9 | Walks allowed per 9 innings |
| SO9 | Strikeouts per 9 innings |
| SO/W | Strikeout-to-walk ratio |
| LOB | Left on base |


In [ ]:
data_folder = 'data/'
team_pitcher_stats = pd.DataFrame()
for file in os.listdir(data_folder):
    if file.startswith('team_pitcher_stats_') and file.endswith('.csv'):
        year = file.split('_')[-1].split('.')[0]
        temp_df = pd.read_csv(os.path.join(data_folder, file))
        temp_df['year'] = int(year)
        team_pitcher_stats = pd.concat([team_pitcher_stats, temp_df], ignore_index=True)
team_pitcher_stats.head(2)

In [ ]:
# remove the rows League Average and NaN teams
team_pitcher_stats = team_pitcher_stats[(~team_pitcher_stats["Tm"].isin(["League Average"])) &
    (team_pitcher_stats["Tm"].notna())
]

### 4.2 Missing Values

In [ ]:
print("\nDF")
df_missing = df.isna().sum()
df_missing = df_missing[df_missing > 0].sort_values(ascending=False)
print(df_missing)

print("\nBATTER_STATS")
batter_missing = batter_stats.isna().sum()
batter_missing = batter_missing[batter_missing > 0].sort_values(ascending=False)
print(batter_missing)

print("\nPITCHER_STATS")
pitcher_missing = pitcher_stats.isna().sum()
pitcher_missing = pitcher_missing[pitcher_missing > 0].sort_values(ascending=False)
print(pitcher_missing)

### 4.3 Data Visualization

In [ ]:
plots = [
    (df, "pitch_type", "df: pitch_type"),
    (df, "events",     "df: events"),
    (df, "stand",      "df: stand"),
    (df, "home_team",  "df: home_team"),
    (batter_stats, "Lev", "batter: Lev"),
    (batter_stats, "Tm",  "batter: Tm"),
    (pitcher_stats, "Lev", "pitcher: Lev"),
    (pitcher_stats, "Tm",  "pitcher: Tm"),
]

sns.set_theme(style="whitegrid")
fig, axes = plt.subplots(2, 4, figsize=(22, 10))
axes = axes.flatten()

colors = ["#5D9BCE"]*4 + ["#79BC8F"]*2 + ["#F4A470"]*2

for i, (ax, (data, col, title)) in enumerate(zip(axes, plots)):
    vc = data[col].value_counts(dropna=False).head(15).reset_index()
    
    sns.barplot(data=vc, x=col, y='count', ax=ax, color=colors[i], edgecolor='w')
    
    ax.set_title(title.upper(), fontweight='bold')
    ax.set_xlabel("") 
    ax.tick_params(axis="x", rotation=45)

plt.tight_layout()
plt.show()

In [ ]:
fig, axes = plt.subplots(3, 4, figsize=(22, 14))
fig.suptitle('Continuous Data', fontsize=24, fontweight='bold', y=1.02)

groups = [
    (df, ["release_speed", "launch_angle", "home_score", "age_bat"], "df", "Blues_d"),
    (batter_stats, ["Age", "PA", "OPS", "HR"], "batter", "Greens_d"),
    (pitcher_stats, ["Age", "IP", "ERA", "SO"], "pitcher", "Oranges_d")
]

for row, (data, cols, label, palette) in enumerate(groups):
    for col_idx, col in enumerate(cols):
        ax = axes[row, col_idx]
        
        if col in data.columns:
            s = pd.to_numeric(data[col], errors="coerce").dropna()
            
            sns.histplot(s, bins=30, kde=True, ax=ax, color=sns.color_palette(palette)[2], edgecolor='w')
            
            ax.set_title(f"{label.upper()}: {col}", fontsize=14, fontweight='semibold')
            ax.set_xlabel(col, fontsize=12)
            ax.set_ylabel("Frequency", fontsize=12)
     

plt.tight_layout()
plt.show()

spiegare che questo dataset permette di fare anche plot e analisi piu avanzate, per esempio heatmap, spyderplot, ecc sia su batter che su pitcher

### 4.4 Player Analysis

#### 4.4.1 Batter Analysis

In [ ]:
events_plot = np.where(
    df.events.isin(['single','double','triple','home_run']),
    df.events,
    'out'
)

opacity = {
    'out': 0.2,
    'single': 0.5,
    'double': 0.7,
    'triple': 0.85,
    'home_run': 1.0
}

In [ ]:
fig = px.scatter(
    df,
    x='launch_speed',
    y='launch_angle',
    color=events_plot,
    category_orders={
        'color': ['out', 'single', 'double', 'triple', 'home_run']
    }
)

for tr in fig.data:
    tr.marker.opacity = opacity[tr.name]

fig.show()

In [ ]:
gif_path = "images/batter.gif"

with open(gif_path, "rb") as f:
    gif_base64 = base64.b64encode(f.read()).decode("ascii")


In [ ]:
df_plot = df.dropna(subset=['launch_speed', 'launch_angle', 'estimated_woba_using_speedangle'])

fig = make_subplots(
    rows=1, cols=2,
    column_widths=[0.45, 0.55],
    specs=[[{"type": "xy"}, {"type": "xy"}]]
)

scatter = px.scatter(
    df_plot, x='launch_speed', y='launch_angle',
    color='estimated_woba_using_speedangle',
    color_continuous_scale='RdYlGn_r',  
    range_color=[0, 1.5],
    labels={'estimated_woba_using_speedangle': 'Est. wOBA Value'}
)

for trace in scatter.data:
    trace.marker.size = 4
    trace.marker.opacity = 0.6
    trace.marker.coloraxis = "coloraxis"
    fig.add_trace(trace, row=1, col=2)

fig.add_layout_image(
    dict(
        source=f"data:image/gif;base64,{gif_base64}",
        xref="x domain", 
        yref="y domain",
        x=0.0,
        y=0.5,
        sizex=1, 
        sizey=1,
        xanchor="left",
        yanchor="middle",
        sizing="contain",
        opacity=1.0,
        layer="below"
    ),
    row=1, col=1
)

fig.update_xaxes(visible=False, range=[0, 1], row=1, col=1)
fig.update_yaxes(visible=False, range=[0, 1], row=1, col=1)

fig.update_xaxes(title_text="Exit Velocity (mph)", range=[40, 120], row=1, col=2)
fig.update_yaxes(title_text="Launch Angle (degrees)", range=[-45, 80], row=1, col=2)

fig.add_hline(y=50, line_dash="dot", line_color="#444", row=1, col=2,
              annotation_text="fly balls", annotation_position="top right")
fig.add_hline(y=25, line_dash="dot", line_color="#444", row=1, col=2,
              annotation_text="line drives", annotation_position="top right")
fig.add_hline(y=10, line_dash="dot", line_color="#444", row=1, col=2,
              annotation_text="ground balls", annotation_position="top right")
fig.add_vline(x=95, line_dash="dash", line_color="#444", row=1, col=2,
              annotation_text="Hard Hit (95+ mph)", annotation_position="top left")

fig.update_layout(
    height=750,
    width=1350,
    template="plotly_white",
    coloraxis=dict(
        colorscale='RdYlGn_r', 
        cmin=0,
        cmax=1.5,
        colorbar=dict(title="Est. wOBA", x=1.02, thickness=15)
    ),
    margin=dict(l=10, r=10, t=80, b=40)
)

fig.show()

source: https://fivethirtyeight.com/features/the-new-science-of-hitting/

#### 4.4.2 Pitcher Analysis

spiegare che esistono diversi tipi di lanci e nel database li abbiamo tutti e li analizziamo raggruppandoli in 4 macrocategorie

<img src="images/types_pitch.png">

come analisi di lanciatori abbiamo metriche come il tipo di lanciamento la velocità e molte altre, questo permette fare plot come i seguenti in cui analizziamo il lanciamento di un determinata persona 

In [ ]:
PITCHER = "Skenes, Paul"

p, s, pl = df["pitch_type"], df["release_speed"], df["pitcher_name"]
m = p.notna() & s.notna() & (pl == PITCHER)

cats = {"Fastball":["FF","SI"], "Changeup":["CH"], "Slider":["SL"], "Curve":["CU"]}
used = [c for v in cats.values() for c in v]
m_used = m & p.isin(used)

In [ ]:
x = np.linspace(65, 102, 600)
dx = x[1] - x[0]
freq = lambda a: gaussian_kde(a)(x) * len(a) * dx

In [ ]:
fig = go.Figure()

all_arr = s[m_used].to_numpy()
if len(all_arr) >= 2:
    fig.add_scatter(x=x, y=freq(all_arr), name="All pitches",
                    line=dict(dash="dash", width=3, color="black"))

for name, codes in cats.items():
    arr = s[m & p.isin(codes)].to_numpy()
    if len(arr) < 2: 
        continue
    fig.add_scatter(x=x, y=freq(arr), name=name, fill="tozeroy")

fig.update_layout(
    title=f"Pitch speed distributions — {PITCHER}",
    xaxis_title="Pitch Speed (mph)",
    yaxis_title="Frequency of Speed"
)
fig.show()


capire effetto del lancio rispetto alla traiettoria della palla

In [ ]:
x_in = df["pfx_x"] * 12
z_in = df["pfx_z"] * 12

fig = go.Figure()

# pitch points
for name, codes in cats.items():
    mm = m & p.isin(codes)
    fig.add_scatter(
        x=-x_in[mm],        
        y=z_in[mm],
        mode="markers",
        name=name,
        marker=dict(size=9, opacity=0.75)
    )

# circles 
t = np.linspace(0, 2*np.pi, 361)
for r in (12, 24):
    fig.add_scatter(
        x=r*np.cos(t), y=r*np.sin(t),
        mode="lines",
        line=dict(dash="dash", width=1, color="gray"),
        showlegend=False
    )

# axes
R = 26
fig.add_shape(type="line", x0=-R, x1=R, y0=0, y1=0, line=dict(width=1, color="gray"))
fig.add_shape(type="line", x0=0, x1=0, y0=-R, y1=R, line=dict(width=1, color="gray"))

# labels
for v in (12, 24):
    fig.add_annotation(x=v, y=0, text=f'{v}"', showarrow=False, font=dict(color="gray"))
    fig.add_annotation(x=0, y=v, text=f'{v}"', showarrow=False, font=dict(color="gray"))

fig.update_layout(
    title=f"{PITCHER} — Movement Profile",
    xaxis=dict(title="Moves toward 3B ", range=[-26, 26], zeroline=False),
    yaxis=dict(title="More Rise  / More Drop ", range=[-26, 26], zeroline=False),
    width=700,
    height=700,
    yaxis_scaleanchor="x",
    yaxis_scaleratio=1
)

fig.show()


In [ ]:
BW = 0.5
pitches = ["FF","SI","SL","CH","CU"]
names  = ["Four-Seam","Sinker","Slider","Change","Curve"]

xx = np.linspace(-2.0, 2.0, 140)
zz = np.linspace(0.0, 5.0, 160)
X, Z = np.meshgrid(xx, zz)
grid = np.vstack([X.ravel(), Z.ravel()])

In [ ]:
fig = make_subplots(rows=1, cols=len(pitches), subplot_titles=names, horizontal_spacing=0.04)

for i, pitch in enumerate(pitches, start=1):
    mm = m_used & (df["pitch_type"] == pitch)
    x = df.loc[mm, "plate_x"].to_numpy()
    z = df.loc[mm, "plate_z"].to_numpy()
    ok = np.isfinite(x) & np.isfinite(z)
    x, z = x[ok], z[ok]
    if x.size < 5:
        continue

    D = gaussian_kde(np.vstack([x, z]), bw_method=BW)(grid).reshape(Z.shape)

    fig.add_trace(
        go.Contour(
            x=xx, y=zz, z=D,
            ncontours=10,
            contours=dict(coloring="heatmap", showlines=True),
            showscale=False, hoverinfo="skip"
        ),
        row=1, col=i
    )

    fig.add_shape(type="rect", x0=-0.83, x1=0.83, y0=1.5, y1=3.5,
                  xref=f"x{i}", yref=f"y{i}", line=dict(color="black", width=2))
    fig.add_shape(type="path",
                  path="M -0.71 0.0 L 0.71 0.0 L 0.35 0.35 L 0.0 0.6 L -0.35 0.35 Z",
                  xref=f"x{i}", yref=f"y{i}", line=dict(color="gray", width=1))

    fig.update_xaxes(range=[-2, 2], showgrid=False, zeroline=False, row=1, col=i)
    fig.update_yaxes(range=[0, 5], showgrid=False, zeroline=False, row=1, col=i)

fig.update_layout(title=f"{PITCHER} — Location KDE ", width=1500, height=360, margin=dict(t=70))
fig.show()


### 4.5 Metric Analysis 

spiegare che abbiamo molte tipologie di metriche: come sono composte, alcune sono semplici altre sono derivate da altre metriche

In [ ]:
batter_stats["Name"] = (
    batter_stats["Name"]
    .astype(str)
    .apply(lambda x: codecs.decode(x, "unicode_escape"))
    .str.encode("latin1")
    .str.decode("utf-8")
)
batter_stats.head()

In [ ]:
batter_stats_filtered = batter_stats[batter_stats['AB'] >= 20].copy()

In [ ]:
# Calculate advanced metrics
batter_stats_filtered['ISO'] = batter_stats_filtered['SLG'] - batter_stats_filtered['BA']  # Isolated Power
batter_stats_filtered['BB_Rate'] = (batter_stats_filtered['BB'] / batter_stats_filtered['PA']) * 100  # Walk Rate %
batter_stats_filtered['K_Rate'] = (batter_stats_filtered['SO'] / batter_stats_filtered['PA']) * 100  # Strikeout Rate %
batter_stats_filtered['HR_Rate'] = (batter_stats_filtered['HR'] / batter_stats_filtered['PA']) * 100  # Home Run Rate %

top_iso = batter_stats_filtered.nlargest(10, 'ISO')[['Name', 'Tm', 'BA', 'SLG', 'ISO', 'HR']]

top_bb = batter_stats_filtered.nlargest(10, 'BB_Rate')[['Name', 'Tm', 'BB_Rate', 'BB', 'OBP']]

In [ ]:
# Violin plots for key metrics
metrics_to_plot = ['BA', 'OBP', 'SLG', 'OPS']

fig = go.Figure()

for i, metric in enumerate(metrics_to_plot):
    fig.add_trace(go.Violin(
        y=batter_stats_filtered[metric],
        name=metric,
        box_visible=True,
        meanline_visible=True,
        fillcolor=['#3498db', '#2ecc71', '#e74c3c', '#f39c12'][i],
        opacity=0.6,
        x0=metric
    ))

fig.update_layout(
    title='Violin Plots of Key Batting Metrics (2024 Season)',
    yaxis_title='Value',
    xaxis_title='Metric',
    height=600,
    plot_bgcolor='#f8f9fa',
    paper_bgcolor='white',
    showlegend=False,
    yaxis=dict(gridcolor='#e1e1e1')
)

fig.show()


In [ ]:
# OBP vs SLG scatter (components of OPS)
median_obp = batter_stats_filtered['OBP'].median()
median_slg = batter_stats_filtered['SLG'].median()

# Create categories based on quadrants
def categorize_player(row):
    if row['OBP'] >= median_obp and row['SLG'] >= median_slg:
        return 'Elite (High OBP & SLG)'
    elif row['OBP'] >= median_obp and row['SLG'] < median_slg:
        return 'Contact (High OBP)'
    elif row['OBP'] < median_obp and row['SLG'] >= median_slg:
        return 'Power (High SLG)'
    else:
        return 'Below Average'

batter_stats_filtered['Category'] = batter_stats_filtered.apply(categorize_player, axis=1)

fig = px.scatter(
    batter_stats_filtered,
    x='SLG',
    y='OBP',
    color='Category',
    size='AB',
    hover_data=['Name', 'Tm', 'HR', 'BA', 'OPS'],
    title='Player Categories: OBP vs SLG Quadrant Analysis',
)

# Add median lines
fig.add_hline(y=median_obp, line_dash="dash", line_color="gray", 
              annotation_text=f"Median OBP: {median_obp:.3f}",
              annotation_position="right")
fig.add_vline(x=median_slg, line_dash="dash", line_color="gray",
              annotation_text=f"Median SLG: {median_slg:.3f}",
              annotation_position="top")

fig.update_layout(
    title_font=dict(size=20, color='#2c3e50', family='Arial Black'),
    height=700,
    plot_bgcolor='#f8f9fa',
    paper_bgcolor='white',
    xaxis=dict(gridcolor='#e1e1e1', title='Slugging Percentage (SLG)'),
    yaxis=dict(gridcolor='#e1e1e1', title='On-Base Percentage (OBP)'),
    legend=dict(title='Player Category', font=dict(size=11))
)

fig.show()


In [ ]:
# BB% vs K% scatter plot
fig = px.scatter(
    batter_stats_filtered,
    x='K_Rate',
    y='BB_Rate',
    size='AB',
    color='OPS',
    hover_data=['Name', 'Tm'],
    color_continuous_scale='Spectral_r',
    title='Plate Discipline: Walk Rate vs Strikeout Rate'
)

# Add quadrant lines
median_k = batter_stats_filtered['K_Rate'].median()
median_bb = batter_stats_filtered['BB_Rate'].median()

fig.add_hline(y=median_bb, line_dash="dash", line_color="gray", 
              annotation_text=f"Median BB%: {median_bb:.1f}%")
fig.add_vline(x=median_k, line_dash="dash", line_color="gray",
              annotation_text=f"Median K%: {median_k:.1f}%")

fig.update_layout(
    title_font=dict(size=20, color='#2c3e50', family='Arial Black'),
    height=700,
    plot_bgcolor='#f8f9fa',
    paper_bgcolor='white',
    xaxis=dict(gridcolor='#e1e1e1', title='Strikeout Rate (%)'),
    yaxis=dict(gridcolor='#e1e1e1', title='Walk Rate (%)'),
    coloraxis_colorbar=dict(title="OPS")
)

fig.show()


In [ ]:
# Create radar chart for top 5 OPS players
top5_players = batter_stats_filtered.nlargest(5, 'OPS')

scaler_radar = MinMaxScaler()

radar_metrics = ['BA', 'OBP', 'SLG', 'ISO', 'BB_Rate', 'HR_Rate']
top5_scaled = top5_players.copy()
top5_scaled[radar_metrics] = scaler_radar.fit_transform(top5_players[radar_metrics])

fig = go.Figure()

colors_radar = ['#e74c3c', '#3498db', '#2ecc71', '#f39c12', '#9b59b6']

for idx, (_, player) in enumerate(top5_scaled.iterrows()):
    fig.add_trace(go.Scatterpolar(
        r=player[radar_metrics].values,
        theta=radar_metrics,
        fill='toself',
        name=player['Name'],
        line_color=colors_radar[idx],
        opacity=0.6
    ))

fig.update_layout(
    polar=dict(
        radialaxis=dict(
            visible=True,
            range=[0, 1]
        ),
        bgcolor='#f8f9fa'
    ),
    title='Top 5 Players Radar Chart',
    height=700,
    showlegend=True,
    paper_bgcolor='white',
    legend=dict(font=dict(size=12))
)

fig.show()

In [ ]:
# 📊 AB vs BA - VARIANCE vs SAMPLE SIZE
# Questo mostra perché serve il Bayesian: alta varianza con pochi AB
fig = px.scatter(
    batter_stats_filtered,
    x='AB',
    y='BA',
    size='H',
    color='PA',
    hover_data=['Name', 'Tm', 'H', 'PA'],
    color_continuous_scale='Viridis',
    title='Batting Average vs At-Bats: The Shrinkage Problem',
    labels={'AB': 'At-Bats (Sample Size)', 'BA': 'Batting Average'}
)

# Add smoothed trend line
from scipy.ndimage import gaussian_filter1d
ab_sorted = batter_stats_filtered.sort_values('AB')
ab_smooth = gaussian_filter1d(ab_sorted['BA'], sigma=10)

fig.add_trace(go.Scatter(
    x=ab_sorted['AB'],
    y=ab_smooth,
    mode='lines',
    line=dict(color='red', width=3, dash='dash'),
    name='Smoothed Trend',
    showlegend=True
))

fig.update_layout(
    height=700,
    plot_bgcolor='#f8f9fa',
    paper_bgcolor='white',
    xaxis=dict(gridcolor='#e1e1e1'),
    yaxis=dict(gridcolor='#e1e1e1'),
    
)

fig.show()

In [ ]:
fig = px.scatter(
    batter_stats_filtered,
    x='PA',
    y='HR_Rate',
    size='HR',
    color='ISO',
    hover_data=['Name', 'Tm', 'HR', 'AB'],
    color_continuous_scale='Hot',
    title='Home Run Rate vs Plate Appearances: Rate Stability',
    labels={'PA': 'Plate Appearances (Exposure)', 'HR_Rate': 'HR Rate (%)'}
)

fig.update_layout(
    height=700,
    plot_bgcolor='#f8f9fa',
    paper_bgcolor='white',
    xaxis=dict(gridcolor='#e1e1e1', title='Plate Appearances (PA)'),
    yaxis=dict(gridcolor='#e1e1e1', title='Home Run Rate (%)'),
    coloraxis_colorbar=dict(title="ISO")
)

fig.show()

# Study I – Moneyball: Team-Level Analysis

## Research Question

Moneyball emphasizes the importance of on-base percentage (OBP) as a measure of a batter's ability to reach base, whether through hits or walks. Historically, batting average (BA) had been the dominant offensive statistic in baseball, rewarding players solely for their ability to hit safely, while ignoring walks. As a result, players with strong plate discipline were systematically undervalued.

The Moneyball philosophy highlighted that reaching base, regardless of how it is achieved, is a fundamental mechanism for generating runs and, ultimately, winning games. In this sense, OBP represents a shift from traditional descriptive to statistics that better capture the underlying processes that determine team success. This insight motivates a formal comparison between OBP and BA in terms of their association with team-level performance outcomes. 
For this reason , our research question is:
**"Is on-base percentage (OBP) a better predictor of team success (measured by runs per game) than batting average (BA) in Major League Baseball?"**

## 5. Bayesian hypothesis testing

### 5.1. H: On-Base Percentage has a larger positive effect on runs per game than Batting Average

The main hypothesis tested in this work is that On-Base Percentagehas a stronger association with team offensive performance, measured by runs per game, which is the primary driver of team wins, than Batting Average. This hypothesis is formalized by comparing the posterior distributions of the corresponding regression coefficients. 

$$ H: \beta_{OBP} > \beta_{BA} $$

where $\beta_{OBP}$ and $\beta_{BA}$ are the regression coefficients for OBP and BA, respectively. We want to asses the probability of this hypothesis being true given the observed data.

$$ P(H | Data) = P(\beta_{OBP} > \beta_{BA} | Data) $$

##### 5.1.1 Priors and Likelihood

we focus on the posterior distribution of the difference between the two coefficient:
$$ \delta = \beta_{OBP} - \beta_{BA} $$
and evaluate the posterior probability that $P(\delta > 0 | Data)$.

the **likelihood function** will be modeled as: $$ y_i \sim Normal(\mu, \sigma) $$ $$ \mu_i = \beta_0 + \beta_{OBP} \cdot OBP_i + \beta_{BA} \cdot BA_i + \epsilon_i$$ where $y_i$ is the runs per game for team $i$

Historically, looking at MLB data, we can see that on average a team scores 4 to 5 points per game, so we consider an average of 4.5 with $\sigma = 0.5$ to include data between 3.5 and 5.5 with approximately 95% probability. As for the priors for $\beta_ {OBP}$ and $\beta_{BA}$, we assume that they are distributions centered at 0 with $\sigma_{\beta} = 1$, positive and negative effects are equally possible a priori.

$$ \beta_0 \sim Normal (4.5, 0.5) $$
$$ \beta_{OBP} \sim Normal(0, 1) $$
$$ \beta_{BA} \sim Normal(0, 1) $$
$$ \epsilon \sim HalfNormal(1) $$

##### 5.1.2 Probabilistic Model

OBP and BA were standardized prior to model fitting to place regression coefficients on a common scale.

In [ ]:
y = team_batter_stats["R/G"].values
team_batter_stats["z_OBP"] = (
    team_batter_stats["OBP"] - team_batter_stats["OBP"].mean()
) / team_batter_stats["OBP"].std()

team_batter_stats["z_BA"] = (
    team_batter_stats["BA"] - team_batter_stats["BA"].mean()
) / team_batter_stats["BA"].std()
x_obp = team_batter_stats["z_OBP"].values
x_ba  = team_batter_stats["z_BA"].values


In [ ]:
with pm.Model() as moneyball_model:

    # Priors
    beta_0    = pm.Normal('beta_0', mu=4.4, sigma=0.5)
    beta_obp  = pm.Normal('beta_OBP', mu=0, sigma=1)
    beta_ba   = pm.Normal('beta_BA', mu=0, sigma=1)
    sigma     = pm.HalfNormal('sigma', sigma=0.5)

    # Linear predictor 
    mu = beta_0 + beta_obp * x_obp + beta_ba * x_ba

    # Likelihood 
    y_obs = pm.Normal('y_obs', mu=mu, sigma=sigma, observed=y)

    # Deterministic
    delta = pm.Deterministic('delta', beta_obp - beta_ba)


In [ ]:
with moneyball_model:
    normal_trace   = pm.sample(chains=4, cores=10)

##### 5.1.3 Inference

In [ ]:
az.plot_trace(normal_trace);

In [ ]:
az.summary(normal_trace, hdi_prob=0.95)

In [ ]:
az.plot_posterior(normal_trace,  var_names="delta", ref_val=0, hdi_prob=0.95);

In [ ]:
az.plot_posterior(normal_trace,  var_names="delta", rope=[-0.05,0.05]);

##### 5.1.4 Prior Sensitivity Analysis

Now let's assume that before seeing the data, even large differences between teams in OBP are unable to explain more or fewer runs per game.

In [ ]:
with pm.Model() as model_prior_tight:

    beta_0   = pm.Normal('beta_0', mu=4.4, sigma=0.5)
    beta_obp = pm.Normal('beta_OBP', mu=0, sigma=0.5)
    beta_ba  = pm.Normal('beta_BA', mu=0, sigma=0.5)
    sigma    = pm.HalfNormal('sigma', sigma=0.5)
    

    mu = beta_0 + beta_obp * x_obp + beta_ba * x_ba

    y_obs = pm.Normal('y_obs', mu=mu, sigma=sigma, observed=y)

    delta = pm.Deterministic('delta', beta_obp - beta_ba)


In [ ]:
with model_prior_tight:
    trace_tight = pm.sample(chains=4, cores=10)

In [ ]:
az.plot_trace(trace_tight);

In [ ]:
az.summary(trace_tight, hdi_prob=0.95)


In [ ]:
az.plot_posterior(trace_tight,  var_names="delta", ref_val=0, hdi_prob=0.95)

##### 5.1.5 Frequentist Comparison

In [ ]:

X = np.column_stack([x_obp, x_ba])
X = sm.add_constant(X)  # intercetta
y = y

ols_model = sm.OLS(y, X).fit()
print(ols_model.summary())

In [ ]:
hypothesis = np.array([0, 1, -1])  # [β0, β_OBP, β_BA]
t_test = ols_model.t_test(hypothesis)

print(t_test)


##### 5.1.6 Posterior Predictive Checks

In [ ]:
beta0_samples   = normal_trace.posterior["beta_0"].values.flatten()
beta_obp_samples = normal_trace.posterior["beta_OBP"].values.flatten()
beta_ba_samples  = normal_trace.posterior["beta_BA"].values.flatten()
sigma_samples    = normal_trace.posterior["sigma"].values.flatten()


In [ ]:
n_draws = 500
idx = np.random.choice(len(beta0_samples), n_draws, replace=False)

y_rep = []

for i in idx:
    mu = beta0_samples[i] + beta_obp_samples[i] * x_obp + beta_ba_samples[i] * x_ba
    y_sim = np.random.normal(mu, sigma_samples[i])
    y_rep.append(y_sim)

y_rep = np.array(y_rep)


In [ ]:
n_ppc_plot = 50
idx_plot = np.random.choice(y_rep.shape[0], n_ppc_plot, replace=False)

fig = go.Figure()

for i in idx_plot:
    fig.add_trace(go.Scatter(
        x=y,
        y=y_rep[i],
        mode="markers",
        marker=dict(size=5, opacity=0.7, color="lightblue"),
        showlegend=False
    ))

fig.add_trace(go.Scatter(
    x=y,
    y=y_rep.mean(axis=0),
    mode="markers",
    marker=dict(size=10, opacity=0.7),
    name="Posterior Predictive Mean"
))

fig.add_trace(go.Scatter(
    x=[y.min(), y.max()],
    y=[y.min(), y.max()],
    mode="lines",
    line=dict(dash="dash", color="black"),
    name="Perfect fit"
))

fig.update_layout(
    title="Posterior Predictive Check: Observed vs Predicted",
    xaxis_title="Observed Runs per Game",
    yaxis_title="Posterior Predictive Mean Runs per Game"
)

fig.show()


## 6. Regression Models

## 7. Hirerchical vs unpooled models

## 8. Model Comparison (WAIC and LOO)

# Study II - Player-Level Analysis